In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

## T1. Actividad productiva
### Indicador: Ratio M/V en tasa de empleo

In [4]:
def actividad_productiva(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de empleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de empleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de empleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de empleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)

    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

## T2. Busqueda de trabajo
### Indicador: Ratio V/M en tasa de desempleo

In [6]:
def busqueda_trabajo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula la actividad productiva, a partir de la proporción de tasas de desempleo entre hombres y mujeres, junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de desempleo (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de desempleo (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de desempleo (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    df_estado = df_temp[(df_temp['ESTADO'] == 1) | (df_temp['ESTADO'] == 2)]
    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 2].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Búsqueda de trabajo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

## T3. Acceso a derechos laborales
### Ratio M/V en tasa de empleo asalariado con descuento jubilatorio

In [8]:
def acceso_derechos_laborales(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción de tasa de registro de la relación laboral entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las tasas de registro (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular tasas de registro (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de tasas de registro (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07H']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a derechos laborales'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

## T4. Participación en el trabajo remunerado
### Ratio M/V en las horas trabajadas remuneradas

In [11]:
def participacion_trabajo_remunerado(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):
    
    """
    Calcula el acceso a derechos laborales, a partir de la proporción en las horas trabajadas remuneradas entre hombres y mujeres junto con los errores asociados.

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tasa (DataFrame): DataFrame con las horas trabajadas remuneradas (M/V) desglosadas por Aglomerado o Provincia. Expresado en % [0-100]
        size (DataFrame): DataFrame con los numeros de observaciones utilizados para calcular las horas trabajadas remuneradas (M/V). Expresado en valores absolutos
        ratio (DataFrame): DataFrame con las proporciones de las horas trabajadas remuneradas (M/V). Expresado en % [0-100]
        error (DataFrame): DataFrame con los errores asociados a las proporciones. Expresado en % [0-100]. 
                            LI: Limite inferior, LS: Limite superior, ME: Margen de error, ER: Error relativo (CV)


    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1)]          # personas ocupadas
    df_estado['PP3F_TOT'] = df_estado['PP3F_TOT'].str.replace(',', '.').astype(float) 

    df_estado['Horas'] = pd.NaT

    mask_999 = (df_estado['PP3E_TOT'] == 999) & (df_estado['PP3F_TOT'] == 999)
    mask_pp3f_999 = (df_estado['PP3F_TOT'] == 999) & (df_estado['PP3E_TOT'] != 999)
    mask_pp3e_999 = (df_estado['PP3E_TOT'] == 999) & (df_estado['PP3F_TOT'] != 999)

    df_estado['Horas'] = np.where(mask_999, pd.NaT,
                                np.where(mask_pp3f_999, df_estado['PP3E_TOT'],
                                        np.where(mask_pp3e_999, df_estado['PP3F_TOT'],
                                                    np.maximum(df_estado['PP3E_TOT'], df_estado['PP3F_TOT'])
                                                )
                                        )
                                )

    df_selected = df_estado[(df_estado['Horas'] > 0) & (~df_estado['Horas'].isna())][['CH04', var, 'Horas', 'PONDERA_new']]
    df_selected['Multiplication'] = df_selected['Horas'] * df_selected['PONDERA_new']
    numerador = df_selected.groupby(['CH04', var])['Multiplication'].sum().unstack(level=0)
    denominador = df_selected.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    tasa = numerador.div(denominador, fill_value=np.nan)    
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[(df_estado['Horas']>0) & (df_estado['Horas']!=np.nan)].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    df_selected['Deviation'] = df_selected['Horas'] - df_selected.groupby(['CH04', var])['Horas'].transform('mean')
    df_selected['Weighted_Deviation'] = df_selected['PONDERA_new'] * df_selected['Deviation']**2
    weighted_variance = df_selected.groupby(['CH04', var])['Weighted_Deviation'].sum() / df_selected.groupby(['CH04', var])['PONDERA_new'].sum()

    weighted_std = np.sqrt(weighted_variance.astype('float')).unstack(level=0)
    weighted_std.columns = ['Varon', 'Mujer']

    standard_error = np.sqrt((weighted_std['Mujer']**2 / size['N_m']) + (weighted_std['Varon']**2 / size['N_v']))

    degrees_of_freedom = size['N_m'] + size['N_v'] - 2
    t = stats.t.ppf((1 + confidence_level) / 2, df=degrees_of_freedom)
    margin_of_error = t * standard_error
    
    ratio = (tasa['Mujer'] / tasa['Varon']).to_frame()
    ratio.rename(columns={0: 'Participacion trabajo remunerado'}, inplace=True)

    lower_bound = tasa['Mujer'] / tasa['Varon'] - margin_of_error
    upper_bound = tasa['Mujer'] / tasa['Varon'] + margin_of_error
    relative_standard_error = margin_of_error / (tasa['Mujer'] / tasa['Varon'])

    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa, size, ratio*100, error

## Funciones auxiliares


In [18]:
# Inactividad
def inactividad(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[df_temp['ESTADO'] == 3].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'] == 3].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Varon']
    p2 = tasa['Mujer']
    n1 = size['N_v']
    n2 = size['N_m']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Inactividad'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [ ]:
# Actividad

def actividad(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    
    numerador = df_temp[df_temp['ESTADO'].isin([1,2])].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)
    denominador = df_temp.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_temp[df_temp['ESTADO'].isin([1,2])].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Actividad'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [ ]:
# T3 bis. 
def acceso_aguinaldo(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07G2']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a aguinaldo'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [21]:
def acceso_obra_social(df, tipo='Aglomerado', base='Individual', confidence_level=0.95):

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]          # personas ocupadas asalariadas
    numerador = df_estado[df_estado['PP07G4']==1].groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    denominador = df_estado.groupby(['CH04', var])['PONDERA_new'].sum().unstack(level=0)

    tasa = numerador.div(denominador, fill_value=np.nan)
    tasa.columns = ['Varon', 'Mujer']

    size = df_estado[df_estado['PP07H']==1].groupby(['CH04', var]).size().unstack(level=0)
    size.columns = ['N_v', 'N_m']

    p1 = tasa['Mujer']
    p2 = tasa['Varon']
    n1 = size['N_m']
    n2 = size['N_v']
    ratio = (p1 / p2).to_frame()
    ratio.rename(columns={0: 'Acceso a OS'}, inplace=True)

    # Calculate the standard error of the proportion ratio
    standard_error = np.sqrt(((1 / n1) * (p1 * (1 - p1))) + ((1 / n2) * (p2 * (1 - p2))))

    z = stats.norm.ppf((1 + confidence_level) / 2)
    margin_of_error = z * standard_error
    lower_bound = p1/p2 - margin_of_error
    upper_bound = p1/p2 + margin_of_error

    relative_standard_error = margin_of_error/(p1/p2)
    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']

    return tasa*100, size, ratio*100, error*100

In [ ]:
def calcular_tabla_poblacion_ocupada_asalariada(df, tipo='Aglomerado', base='Individual'):

    """
    Calcula la tabla de .... 

    Args:
        df (DataFrame): DataFrame de entrada que contiene los datos de la EPH.
        tipo (str): Tipo de encuesta de la EPH, 'Aglomerado' o 'Urbano'. Por defecto: 'Aglomerado'.
        base (str): Tipo de base de la EPH, 'Individual' o 'Hogar'. Por defecto: 'Individual'.
        confidence_level (float): Nivel de confianza para el cálculo de errores. Por defecto: 0.95.

    Returns:
        tabla (DataFrame): DataFrame con la tabla_pob_ocupada_asalariada

    """
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA_new']):
        raise ValueError("El DataFrame de entrada debe contener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA_new'")

    if tipo == 'Aglomerado':
        var = 'AGLOMERADO'
    elif tipo == 'Urbano':
        var = 'PROVINCIA'
    
    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] == 3)]
    
    resultados = df_estado.groupby(['CH04', 'PP07H', 'PP07I'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)
    resultados.columns = ['Varones', 'Mujeres']
    resultados = resultados.reset_index()
    
    resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']
    resultados['Combinación'] = resultados.apply(lambda row: f"PP07H == {row['PP07H']} y PP07I == {row['PP07I']}", axis=1)
    
    resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)
    
    resultados.loc[:, '% Varones'] = resultados['Varones'] / resultados.loc['Total Col', 'Varones'] * 100
    resultados.loc[:, '% Mujeres'] = resultados['Mujeres'] / resultados.loc['Total Col', 'Mujeres'] * 100
    
    resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]
    
    tabla_pob_ocupada_asalariada = resultados.style.format({
        'Varones': '{:,.0f}',
        '% Varones': '{:,.2f}',
        'Mujeres': '{:,.0f}',
        '% Mujeres': '{:,.2f}',
        'Total Fila': '{:,.0f}'
    })
    
    return tabla_pob_ocupada_asalariada


In [ ]:
## pluriempleo

# df_temp = df_people_pool.query('CH06 >= 16 & CH06 < 65')
# df_estado = df_temp[(df_temp['ESTADO'] == 1) & (df_temp['CAT_OCUP'] != 1)& (df_temp['CAT_OCUP'] != 4)]          # personas ocupadas cuentapropistas, obreros o empleados

# # Filtrar y contar las combinaciones por género
# resultados = df_estado.groupby(['CH04', 'CAT_OCUP', 'PP03C'])['PONDERA_new'].sum().unstack(level='CH04', fill_value=0)

# resultados.columns = ['Varones', 'Mujeres']
# resultados = resultados.reset_index()

# resultados['Total Fila'] = resultados['Varones'] + resultados['Mujeres']
# resultados['Combinación'] = resultados.apply(lambda row: f"CAT_OCUP == {row['CAT_OCUP']}, PP03C == {row['PP03C']}", axis=1)

# resultados.loc['Total Col'] = resultados.sum(numeric_only=True, axis=0)
# resultados.loc[:, '% Varones'] = resultados['Varones']/resultados.loc['Total Col', 'Varones'] * 100
# resultados.loc[:, '% Mujeres'] = resultados['Mujeres']/resultados.loc['Total Col', 'Mujeres'] * 100

# resultados = resultados[['Combinación', 'Varones', '% Varones', 'Mujeres', '% Mujeres', 'Total Fila']]

# # Aplicar formato a los valores en la tabla
# tabla_pob_ocupada = resultados.style.format({
#     'Varones': '{:,.0f}',
#     '% Varones': '{:,.2f}',
#     'Mujeres': '{:,.0f}',
#     '% Mujeres': '{:,.2f}',
#     'Total Fila': '{:,.0f}'
# })

# tabla_pob_ocupada

# [P21 mean M y V, TOT_P12 mean M y V]